In [1]:
# import sys
import time
import random
from configparser import ConfigParser
import subprocess

# config = ConfigParser()
# CARLA_PATH = config.get("main", "CARLA_PATH")
# # Enable import of 'carla'
# sys.path.append(CARLA_PATH + "PythonAPI/carla/dist/carla-0.9.9-py3.6-linux-x86_64.egg")

import carla

# Get client to interact with CARLA server
client = carla.Client('127.0.0.1', 2000)

# Get current CARLA world
world = client.get_world()
Map = world.get_map()

In [2]:
# set town and weather
subprocess.call("python ~/carla/PythonAPI/util/config.py --m Town04 --weather ClearNoon", shell=True)

#TODO: Set camera view

0

-----------STEPS TO CREATE AN AUTOPILOT VEHICLE---------------
1. Instantiate a traffic manager, set tm params. 
2. Get vehicle blueprints
3. Get spawn points on the desired lane
4. Associate blueprint to vehicles and spawn (SpawnActor)
5. Set autopilot on the vehicle and register to TM (SetAutopilot)

In [3]:
# Get Traffic Manager instance
tm = client.get_trafficmanager(8000)

In [77]:
# set the spectator
spectator = world.get_spectator()
trans = carla.Transform(carla.Location(x=-360.875458, y=-18.428667, z=46.781178), carla.Rotation(pitch=-39.588058, yaw=41.867565, roll=0.000029))
spectator.set_transform(trans)

In [70]:
# First lane waypoint XYZ
town04_highway = {'X':-360, 'Y':30, 'Z':0, 
                  "distance_bwn_waypoints":8,
                  "total_non_ego_vehicles":3,
                  "max_vehicles_in_front":2,
                  "target_speed":15,
                 }

# Get vehicle blueprints, mustang for non-ego, tesla for ego :D
non_ego_vehicle_blueprint = world.get_blueprint_library().filter('vehicle.mustang.mustang')[0]
ego_vehicle_blueprint = world.get_blueprint_library().filter('vehicle.tesla.model3')[0]

# Get spawn points
# Get a list of waypoints 1 m apart on the desired lane
config = town04_highway # change to get lane following scenario in another location
loc = carla.Location(x=config['X'], y=config['Y'], z=config['Z'])
first_waypoint = Map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
waypoint_list = first_waypoint.next_until_lane_end(config["distance_bwn_waypoints"])
spawn_points_list = [x.transform for x in waypoint_list]
for point in spawn_points_list: point.location.z += 0.4

# non-ego vehicle spawn points
non_ego_spawn_points = [(spawn_points_list[i], spawn_points_list[i].get_forward_vector()) \
                         for i in range(1,1+config["total_non_ego_vehicles"])]
# non_ego_spawn_points = [spawn_points_list[i] for i in range(1,1+config["total_non_ego_vehicles"])]
vectors = [x.get_forward_vector() for x in spawn_points_list[1:5]]
ego_spawn_point = (spawn_points_list[0], spawn_points_list[0].get_forward_vector())

In [71]:
# destroy all old actors
all_actors = world.get_actors().filter("vehicle*")
client.apply_batch([carla.command.DestroyActor(x) for x in all_actors])
world.tick()
print(world.get_actors().filter("vehicle*"))

[]


In [72]:
# defining a few commands
SpawnActor = carla.command.SpawnActor
SetAutopilot = carla.command.SetAutopilot
FutureActor = carla.command.FutureActor
ApplyVelocity = carla.command.ApplyVelocity

#------------SETTING THE WORLD AND TM TO SYNCRONOUS MODE--------------
no_rendering_mode=False
synchronous_mode=True
fixed_delta_seconds=0.03
world.apply_settings(carla.WorldSettings(no_rendering_mode=no_rendering_mode,
                                         synchronous_mode=synchronous_mode,
                                         fixed_delta_seconds=fixed_delta_seconds))
tm.set_synchronous_mode(synchronous_mode)
tm.set_hybrid_physics_mode(True)
world.tick()

batch = []
for i in range(config["total_non_ego_vehicles"]):
    batch.append(SpawnActor(non_ego_vehicle_blueprint, non_ego_spawn_points[i][0])
        .then(ApplyVelocity(FutureActor, non_ego_spawn_points[i][1] * config["target_speed"]))
        .then(SetAutopilot(FutureActor, True, tm.get_port())))

batch.append(SpawnActor(ego_vehicle_blueprint, ego_spawn_point[0])
        .then(ApplyVelocity(FutureActor, ego_spawn_point[1] * config["target_speed"]))
        .then(SetAutopilot(FutureActor, True, tm.get_port())))

In [73]:
vehicle_list = []
for response in client.apply_batch_sync(batch, False):
    if response.error:
        print(response.error)
    else:
        vehicle_list.append(response.actor_id)
if synchronous_mode:
    world.tick()

all_actors = world.get_actors().filter("vehicle*")
for actor in all_actors:
    tm.auto_lane_change(actor, False)
    tm.ignore_vehicles_percentage(actor, 60)
    actor.set_simulate_physics(False)
    
tm.set_hybrid_physics_mode(True)
print(all_actors)

[Actor(id=330, type=vehicle.mustang.mustang), Actor(id=331, type=vehicle.mustang.mustang), Actor(id=332, type=vehicle.mustang.mustang), Actor(id=333, type=vehicle.tesla.model3)]


In [6]:
st = time.time()
past_time = world.get_snapshot().timestamp.elapsed_seconds
past_frame = world.get_snapshot().timestamp.frame

ctr = 0

while True:
    ct = time.time()
#     time.sleep(0.02)
    if ct - st > 1:
        print(ctr)
        ctr = 0
        st = ct
#         actor_vels = [actor.get_velocity().x for actor in all_actors]
        curr_frame = world.get_snapshot().timestamp.frame
        curr_time  = world.get_snapshot().timestamp.elapsed_seconds
        print( curr_frame - past_frame, ' ', curr_time - past_time, ' ', actor_vels)
        past_frame = curr_frame
        past_time  = curr_time
    ctr+=1
#     print('\r  ',ctr,end='  ')
#     world.tick()

5349852
23   1.1500000171363354   [0.0, 0.0, 0.0]
5359219
22   1.1000000163912773   [0.0, 0.0, 0.0]
5357766
23   1.1500000171363354   [0.0, 0.0, 0.0]
5426079
22   1.1000000163912773   [0.0, 0.0, 0.0]
5450609
23   1.1500000171363354   [0.0, 0.0, 0.0]
5437710
25   1.2500000186264515   [0.0, 0.0, 0.0]
5449983
22   1.1000000163912773   [0.0, 0.0, 0.0]


KeyboardInterrupt: 

In [ ]:
# impulse = 10000
# i = 0
# for actor_id in vehicle_list:
#     impulse_vector = vectors[i]*impulse
#     world.get_actor(actor_id).add_impulse(impulse_vector)
#     i+=1

In [217]:
print(world.get_snapshot().timestamp)

Timestamp(frame=1206725,elapsed_seconds=13822.411183,delta_seconds=0.050000,platform_timestamp=279098.046813)


In [10]:
import numpy as np
np.random.randint(5,9)

7

In [11]:
a = [1,2,3]
b = a.pop(1)
a

[1, 3]

In [12]:
b

2